In [1]:
%load_ext lab_black
%cd ../../..

/home/shim/cev/dl/log-analytics


In [2]:
import torch
import numpy as np
import pandas as pd
import pickle

In [3]:
df = pd.read_csv("data/ori/test.csv")

In [4]:
tdf = pd.read_csv("data/ori/train.csv")

In [5]:
deck1 = torch.load("results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05_4-deck1.pth")
deck2 = torch.load("results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05_4-deck2.pth")

In [6]:
deck1["feat"].shape, deck2["feat"].shape

(torch.Size([472972, 768]), torch.Size([1418916, 768]))

In [7]:
deck1.keys(), deck2.keys()

(dict_keys(['fcfeat', 'tlevel', 'fclevel', 'feat']),
 dict_keys(['fcfeat', 'fclevel', 'feat']))

In [8]:
deck1["feat_"] = deck1["feat"].cuda()
deck2["feat_"] = deck2["feat"].cuda()

In [9]:
j = 0
shown = False

In [10]:
dist_ = torch.norm(deck1["feat_"] - deck2["feat_"][j, None], p=None, dim=1)
values_, indices_ = dist_.topk(4, largest=False)
tlevels = deck1["tlevel"][indices_]
print("values:", values_)
print("indices:", indices_)
print("id:", deck1["tlevel"][indices_])
j += 1

values: tensor([0., 0., 0., 0.], device='cuda:0')
indices: tensor([406, 215, 521, 748], device='cuda:0')
id: tensor([0, 0, 0, 0])


In [ ]:
while shown or (deck1["tlevel"][indices_[0]] in [0, 1, 3, 5] and values_[0] < 0.3):
    # while shown or (deck1["tlevel"][indices_[0]] in [0, 1, 3, 5]):
    shown = False

    dist_ = torch.norm(deck1["feat_"] - deck2["feat_"][j, None], p=None, dim=1)
    values_, indices_ = dist_.topk(4, largest=False)
    tlevels = deck1["tlevel"][indices_]
    # print("values:", values)
    # print("indices:", indices)
    # print("id:", deck1["tlevel"][indices])
    j += 1

In [ ]:
print("*  Submit :", df.full_log[j - 1])
print("----------------------------------------------------------------------------------------")
print("*   index :", j - 1)
print("*    dist :", " ".join(map(lambda x: f"{x:.4f}", values_.cpu().tolist())))
print("* fc-level:", deck2["fclevel"][j - 1])
print("----------------------------------------------------------------------------------------")

for idx in indices_:
    print("* Level", deck1["tlevel"][idx].item(), ":", tdf.full_log[idx.item()])
    print("----------------------------------------------------------------------------------------")

shown = True

---

0.91 나온 녀석도 어떤건 0.3보다 커도 level7이 아닌 것 같은게 있었지만, 그냥 0.3으로 통일해서 냈을 때가 가장 점수가 높았다.

그래도 level5는 높게 잡아줘야 하지 않을까? 아니면 길이랑 상관관계가 있을 것 같기도?

```
threshold = 0.3 + 0.2 * (token_length / 256 - 1) ??
```

길이가 꽉차면 0.5, 짧을 수록 0.1 ??

아니멸 fc_level 때문이었는지도 모르고